In [6]:
import bittensor as bt

In [20]:
sub = bt.subtensor('local')

2024-07-17 11:27:22.518 |       INFO       | Connected to local network and ws://127.0.0.1:9944.


In [7]:
sub.get_subnets()

[0, 1, 2, 3, 4, 5]

In [14]:
alice = bt.wallet('alice')
bob = bt.wallet('bob')

In [11]:
# Add stake.
sub = bt.subtensor('local')
alice = bt.wallet()
keypair = bt.Keypair.create_from_uri('//Alice')
alice.set_coldkey(keypair=keypair, encrypt=False, overwrite=True)
alice.set_coldkeypub(keypair=keypair, encrypt=False, overwrite=True)
alice.set_hotkey(keypair=keypair, encrypt=False, overwrite=True)
call = sub.substrate.compose_call(
    call_module="SubtensorModule",
    call_function="add_stake",
    call_params={
        "hotkey": alice.hotkey.ss58_address,
        "netuid": 2,
        "amount_staked": 1_000_000_000,
    },
)
extrinsic = sub.substrate.create_signed_extrinsic(
    call=call, keypair=alice.coldkey
)
response = sub.substrate.submit_extrinsic(
    extrinsic,
    wait_for_finalization=True,
)
response.process_events()
if not response.is_success:
    print(response.error_message)
else:
    print ('success')

2024-07-17 11:35:47.633 |       INFO       | Connected to local network and ws://127.0.0.1:9944.
success


In [6]:
# Create networks
import bittensor as bt
sub = bt.subtensor('local')
alice = bt.wallet()
keypair = bt.Keypair.create_from_uri('//Alice')
alice.set_coldkey(keypair=keypair, encrypt=False, overwrite=True)
alice.set_coldkeypub(keypair=keypair, encrypt=False, overwrite=True)
alice.set_hotkey(keypair=keypair, encrypt=False, overwrite=True)
call = sub.substrate.compose_call(
    call_module="SubtensorModule",
    call_function="register_network",
)
extrinsic = sub.substrate.create_signed_extrinsic(
    call=call, keypair=alice.coldkey
)
response = sub.substrate.submit_extrinsic(
    extrinsic,
    wait_for_finalization=True,
)
response.process_events()
if not response.is_success:
    print(response.error_message)
else:
    print ('success')

2024-07-17 11:33:26.494 |       INFO       | Connected to local network and ws://127.0.0.1:9944.
success


In [9]:
# Register keys
import bittensor as bt
sub = bt.subtensor('local')
alice = bt.wallet()
keypair = bt.Keypair.create_from_uri('//Alice')
alice.set_coldkey(keypair=keypair, encrypt=False, overwrite=True)
alice.set_coldkeypub(keypair=keypair, encrypt=False, overwrite=True)
alice.set_hotkey(keypair=keypair, encrypt=False, overwrite=True)
call = sub.substrate.compose_call(
    call_module="SubtensorModule",
    call_function="burned_register",
    call_params={
        "hotkey": alice.hotkey.ss58_address,
        "netuid": 2,
    },
)
extrinsic = sub.substrate.create_signed_extrinsic(
    call=call, keypair=alice.coldkey
)
response = sub.substrate.submit_extrinsic(
    extrinsic,
    wait_for_finalization=True,
)
response.process_events()
if not response.is_success:
    print(response.error_message)
else:
    print ('success')

2024-07-17 11:35:08.258 |       INFO       | Connected to local network and ws://127.0.0.1:9944.
success


In [10]:
sub.get_all_neurons_for_pubkey(alice.hotkey.ss58_address)

[NeuronInfo(hotkey='5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', coldkey='5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', uid=0, netuid=2, active=True, stake=τ0.000000000, stake_dict={'5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY': τ0.000000000}, total_stake=τ0.000000000, rank=0.0, emission=0.0, incentive=0.0, consensus=0.0, trust=0.0, validator_trust=0.0, dividends=0.0, last_update=2740, validator_permit=False, weights=[], bonds=[], pruning_score=0, prometheus_info=PrometheusInfo(block=0, version=0, ip='0.0.0.0', port=0, ip_type=0), axon_info=AxonInfo( /ipv0/0.0.0.0:0, 5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY, 5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY, 0 ), is_null=False)]